In [1]:
import sys, os; sys.path.append('..')
import pyzx as zx
import random
import math
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [2]:
g = zx.Circuit.load('../circuits/Fast/hwb6.qc').to_graph()
zx.d3.draw(g)
print(zx.tcount(g))

105


In [3]:
zx.simplify.full_reduce(g)
zx.d3.draw(g)
print(zx.tcount(g))

75


In [4]:
g.apply_state("++---+-")
g.apply_effect("+011-1-")
zx.simplify.full_reduce(g)
print(zx.tcount(g))
zx.drawing.arrange_scalar_diagram(g)
zx.d3.draw(g)

33


In [13]:
def decomp_search(g, depth=0, num_tries=600):
    if zx.tcount(g) >= 6:
        prev_best = -1
        prev_best_gsum = None
        tries = 0
        total_tries = 0
        while True:
            total_tries += 1
            gsum = zx.simulate.replace_magic_states(g, pick_random=True)
            gsum.reduce_scalar()
            approx_terms = sum(math.ceil(2**(0.468 * zx.tcount(g))) for g in gsum.graphs)
            if prev_best == -1 or prev_best > approx_terms:
                prev_best = approx_terms
                prev_best_gsum = gsum
                tries = 1
            else:
                tries += 1
            if tries >= num_tries:
                break
        gsum = prev_best_gsum
        vals = 0
        terms = 0
        for g1 in gsum.graphs:
            t,tt,v = decomp_search(g1, depth+1, num_tries=math.ceil(num_tries/2))
            terms += t
            total_tries += tt
            vals += v
        return (terms, total_tries, vals)
    else:
        return (2 ** math.ceil(zx.tcount(g)/2), 1, g.to_matrix())

Expected summands for s.o.t.a. exact decomposition and approximate decomposition with norm distance $< 0.1$.

In [9]:
(math.ceil(2**(0.468 * zx.tcount(g))), math.ceil(((0.1)**(-2))*2**(0.228 * zx.tcount(g))))

(44577, 18406)

In [14]:
random.seed(300)
print(decomp_search(g))

(49, 3370, array([[-0.0625-0.j]]))


In [ ]:
random.seed(400)
print(decomp_search(g))
random.seed(500)
print(decomp_search(g))

In [70]:
zx.simulate.calculate_path_sum(g)

(-0.06250000000000155+1.4561767220704996e-16j)

In [95]:
math.log(math.cos(math.pi / 8)**-2, 2)

0.22844669683638807

In [98]:
def eff(x,y): return math.log(y,2)/x

In [100]:
[eff(x,y) for x,y in [(1,2),(2,3),(3,4),(4,5),(5,6),(6,8),(7,14)]]

[1.0,
 0.7924812503605781,
 0.6666666666666666,
 0.5804820237218405,
 0.5169925001442313,
 0.5,
 0.5439078460082291]